## Разбор результатов тестов учащихся

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

Загрузка файла с данными об успеваемости из подпапки *data*. Так как исходный файл очень большой, загружаются только первые 10 млн. записей.

In [2]:
train_data = pd.read_csv('data/train.csv', nrows=10000000)

Определение столбцов, содержащихся в файле с данными.

In [7]:
train_data.columns

Index(['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation'],
      dtype='object')

Просмотр ответов на все вопросы тестов, данными учащимся с id=115.

In [14]:
train_data.loc[train_data['user_id'] == 115]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
5,5,157063,115,156,0,5,2,1,5000.0,False
6,6,176092,115,51,0,6,0,1,17000.0,False
7,7,194190,115,50,0,7,3,1,17000.0,False
8,8,212463,115,7896,0,8,2,1,16000.0,False
9,9,230983,115,7863,0,9,0,1,16000.0,False


Чтобы определить успеваемость каждого учащегося, исходная таблица разбивается по *user_id* на отдельные таблицы. 

In [3]:
users = [user_data for user, user_data in train_data.groupby('user_id')]

По каждому учащемуся формируется сводная таблица, содержащая количество дававшихся вопросов, количество правильных ответов, потраченное время на каждый вопрос, количество вопросов с пояснениями.

In [73]:
user_ids = [user['user_id'].values[0] for user in users]
user_quest_count = [user.shape[0] for user in users]
user_corr_answers = [user['answered_correctly'].sum() for user in users]
user_quest_elap_time = [user['prior_question_elapsed_time'].mean(skipna=True) for user in users]
user_quest_had_expl = [user['prior_question_had_explanation'].value_counts().get(True, 0) for user in users]

user_test_stat_df = pd.DataFrame({'user_id': user_ids, 'question_count': user_quest_count,
    'correct_answers': user_corr_answers, 'prior_question_elapsed_time': user_quest_elap_time,
    'questions_with_prior_explanations': user_quest_had_expl})

Для объективной оценки в сводную таблицу добавляются столбцы с процентами правильных ответов и вопросов с пояснениями.

In [92]:
user_test_stat_df['correct_answers_pc'] = user_test_stat_df['correct_answers'] / user_test_stat_df['question_count'] * 100.0
user_test_stat_df['questions_with_prior_explanations_pc'] = user_test_stat_df['questions_with_prior_explanations'] / user_test_stat_df['question_count'] * 100.0

Просмотр первых строк получившейся сводной таблицы.

In [93]:
user_test_stat_df.head()

,user_id,question_count,correct_answers,prior_question_elapsed_time,questions_with_prior_explanations,correct_answers_pc,questions_with_prior_explanations_pc
0,115,46,32,19933.311111,6,69.565217,13.043478
1,124,30,7,18793.000000,0,23.333333,0.000000
2,2746,20,10,18055.555556,11,50.000000,55.000000
3,5382,128,81,36048.387097,113,63.281250,88.281250
4,8623,112,67,26107.407407,96,59.821429,85.714286


Просмотр статистики по сводной таблице.

In [98]:
user_test_stat_df.describe()

,user_id,question_count,correct_answers,prior_question_elapsed_time,questions_with_prior_explanations,correct_answers_pc,questions_with_prior_explanations_pc
count,3.949100e+04,39491.000000,39491.000000,39479.000000,39491.000000,39491.000000,39491.000000
mean,1.081927e+08,253.222253,158.568509,23894.468107,224.242359,53.098934,54.836697
std,6.270965e+07,731.820282,498.826812,8052.807093,703.932723,16.073574,35.110165
min,1.150000e+02,1.000000,-2.000000,0.000000,0.000000,-1.307190,0.000000
25%,5.366920e+07,30.000000,11.000000,19109.478540,6.000000,42.307692,22.500000
50%,1.079776e+08,40.000000,22.000000,22719.598425,20.000000,55.000000,63.039400
75%,1.629231e+08,157.000000,91.000000,27323.529412,130.000000,64.796994,88.000000
max,2.167479e+08,15871.000000,11605.000000,230833.333333,15381.000000,100.000000,100.000000


Из статистики видно, что в среднем доля вопросов, на которые учащиеся отвечают правильно, составляет чуть больше половины от всех дававшихся им вопросов. Среднее время, которое учащиеся тратили на один вопрос, составляет 24 секунды.

Вывод таблицы учащихся с наилучшими результатами, которые набрали от 95% и которым давалось от 15 вопросов и больше.

In [94]:
user_test_stat_df.loc[(user_test_stat_df['question_count'] >= 15) & (user_test_stat_df['correct_answers_pc'] >= 95.0)]

,user_id,question_count,correct_answers,prior_question_elapsed_time,questions_with_prior_explanations,correct_answers_pc,questions_with_prior_explanations_pc
543,3126556,30,29,13804.448276,0,96.666667,0.000000
1339,7528236,31,30,17097.066667,0,96.774194,0.000000
1392,7788970,30,29,15356.172414,0,96.666667,0.000000
3510,19087497,31,30,17900.000000,19,96.774194,61.290323
5055,27630548,21,20,12750.000000,9,95.238095,42.857143
6481,35662893,91,90,19420.355556,0,98.901099,0.000000
7934,43283008,30,29,12344.827586,0,96.666667,0.000000
8306,45312560,30,30,5517.137931,0,100.000000,0.000000
8763,48014622,30,29,19482.655172,0,96.666667,0.000000
8779,48102728,41,41,40200.000000,33,100.000000,80.487805


Разбор показывает, что учащихся с очень высокими результатами крайне мало среди общего числа учащихся.
Также среди таких учащихся примерно одинаковое количество получавших вопросы с пояснениями и вопросы без пояснений.
Можно предположить, что наличие пояснений к вопросам слабо влияет на результаты тестирования у самых сильных учащихся.